# Introduction to `starry`
Here we'll briefly go over the basics of spherical harmonic maps with `starry`. Make sure to also check out the [tutorials page](https://rodluger.github.io/starry/tutorials.html) for more examples on how to use the code.

## Imports & setup

In [ ]:
%matplotlib inline

In [ ]:
%run workshop_utils/notebook_setup.py

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## Introduction
A *spherical harmonic map* is a vector of spherical harmonic coefficients, indexed by increasing degree $l$ and order $m$:

$y = \{Y_{0,0}, \, Y_{1,-1}, \, Y_{1,0}, \, Y_{1,1} \, Y_{2,-2}, \, Y_{2,-1}, \, Y_{2,0} \, Y_{2,1}, \, Y_{2,2}, \, ...\}$.

For reference, here's what the first several spherical harmonic degrees look like:

![](img/ylms.png)

Each row corresponds to a different degree $l$, starting at $l = 0$. Within each row, the harmonics extend from order $m = -l$ to order $m = l$. Together, the spherical harmonics form a complete basis set on the sphere, so in principle *any surface map* can be expanded in terms of these functions.

As an example, suppose we have the following map vector:

```
y = [1.00,  0.22,  0.19,  0.11,  0.11,  0.07,  -0.11, 0.00,  -0.05,
     0.12,  0.16,  -0.05, 0.06,  0.12,  0.05,  -0.10, 0.04,  -0.02,
     0.01,  0.10,  0.08,  0.15,  0.13,  -0.11, -0.07, -0.14, 0.06,
     -0.19, -0.02, 0.07,  -0.02, 0.07,  -0.01, -0.07, 0.04,  0.00]
```

This is how much each spherical harmonic is contributing to the sum:

![](img/ylms_earth.png)

If we add up all of the terms, we get the following image:

![](img/earth5.png)

which is the $l = 5$ spherical harmonic expansion of a map of the Earth! South America is to the left and Africa is toward the top right. It might still be hard to see, so here's what we would get if we carried the expansion up to degree $l = 20$:

![](img/earth20.png)

Before we dig into the code, one final cool thing about spherical harmonics is that they can be rotated, which makes them ideal for modeling planets, stars, and other celestial bodies:

![](img/earth.gif)

## Using `starry`
OK, now that we've introduced the spherical harmonics, let's look at how we can use `starry` to model some celestial bodies.

The first thing we should do is import `starry` and instantiate a `Map` object. This is the simplest way of creating a spherical harmonic map. The `Map` object takes a few arguments, the most important of which is `lmax`, the highest degree of the spherical harmonics used to describe the map. Let's create a fifth-order map:

In [ ]:
import starry
map = starry.Map(lmax=5)

The `y` attribute of the map stores the spherical harmonic coefficients. We can see that our map is initialized to a constant map: 

In [ ]:
map.y

By default, the $Y_{0,0}$ coefficient is set to unity, and all other coefficients are set to zero. Our map is therefore just the first spherical harmonic, which if you scroll up you'll see is that constant dark blue disk at the top of the first figure. We can also quickly visualize the map by calling the `show` method:

In [ ]:
map.show()

## Setting map coefficients

Setting spherical harmonic coefficients is extremely easy: we can assign values directly to the map instance itself. Say we wish to set the coefficient of the spherical harmonic $Y_{5, -3}$ to $-2$. We simply run

In [ ]:
map[5, -3] = -2

We can check that the spherical harmonic vector (which is a flattened version of the image we showed above) has been updated accordingly:

In [ ]:
map.y

And here's what our map now looks like:

In [ ]:
map.show()

Just for fun, let's set two additional coefficients:

In [ ]:
map[5, 0] = 2
map[5, 4] = 1

In [ ]:
map.show()

One last note on visualizing maps: you can quickly animate the map by setting the map's rotation axis,

In [ ]:
map.axis = [0, 1, 0]

(in this case, $\hat{y}$) and calling the `animate` method:

In [ ]:
# You can also call `map.animate()`, but the routine in 
# the `workshop_utils` module is better for Jupyter notebooks!
from workshop_utils import animate
animate(map)

## ★ Exercise 1

Play around with setting different combinations of spherical harmonic coefficients, changing the map's rotation axis, and calling the `animate` method to see how the map changes.

## Computing phase curves
We've seen how to instantiate and manipulate maps, but what we really would like to do is compute light curves. So let's get to it. We'll use the same map as before:

In [ ]:
map.reset()
map[5, -3] = -2
map[5, 0] = 2
map[5, 4] = 1
map.show()

We will compute one full cycle of the phase curve by varying `theta` between 0 and 360 degrees. For simplicity, let's pick $\hat{y}$ as our rotation axis.

In [ ]:
theta = np.linspace(0, 360, 100, endpoint=False)
map.axis = [0, 1, 0]

Computing the phase curve with `starry` is super easy:

In [ ]:
F = map.flux(theta=theta)

Plot the results:

In [ ]:
plt.plot(theta, F)
plt.xlabel(r'$\theta$ (degrees)')
plt.ylabel(r'Flux');

## ★ Exercise 2

Why is there no phase curve?! Set a few other spherical harmonic coefficients to see if you can get a phase curve, and try to figure out the general rule for the *null space* of the phase curve problem.

## ★ Exercise 3

Plot the phase curve of your new map for a few different values of the axis of rotation.

## Computing occultation light curves

Finally, we're ready to discuss occultation light curves! Let's occult the following map with a small occultor and compute the resulting light curve.

In [ ]:
map.reset()
map[2, -1] = -0.3
map[3, 0] = 0.1
map[4, 1] = 0.3
map.show()

Consider an occultor of radius $r_o = 0.1$ moving along the $+x$ direction at $y_o = -0.5$. Suppose that the occulted body rotates by $30^\circ$ about the $y$-axis in the meantime. Let's define our variables:

In [ ]:
npts = 1000
ro = 0.1
xo = np.linspace(-3, 3, npts)
yo = -0.5
theta = np.linspace(0, 30, npts, endpoint=False)
map.axis = [0, 1, 0]

Computing the occultation light curve is now easy:

In [ ]:
F = map.flux(theta=theta, xo=xo, yo=yo, ro=ro)

Here's the rather odd-looking light curve:

In [ ]:
plt.plot(theta, F, lw=2)
plt.xlabel(r'$\theta$ (degrees)')
plt.ylabel(r'Flux');

## ★ Exercise 4

Interpret the various features of the light curve. Then vary the occultor size and trajectory, overplot the resulting light curves, and interpret the results.